In [ ]:
import os
os.environ["IMAGEIO_FFMPEG_EXE"] = "/Users/Linshaoqian/Downloads/ffmpeg"

import moviepy.editor as mp
# 為moviepy设置ffmpeg路徑

from pydub import AudioSegment

from google.cloud import speech

# 設置 Google Application Credentials 環境變量
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "gcpai.json"

In [ ]:
# 將影片檔案轉換為音檔
video = mp.VideoFileClip("周處除三害-77.mp4")
audio = video.audio
audio.write_audiofile("converted_audio-77.wav")

In [ ]:
# 读取音频文件
audio = AudioSegment.from_file("converted_audio-77.wav")

# 将立体声音频转换为单声道
mono_audio = audio.set_channels(1)

# 保存转换后的音频
mono_audio.export("77.wav", format="wav")

In [ ]:
# 使用Google Speech-to-Text進行語音辨識
client = speech.SpeechClient()

# 使用GCS上的音檔文件
gcs_uri = "gs://movie77/77.wav"
audio = speech.RecognitionAudio(uri=gcs_uri)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=44100,  # 设置与音频文件中相匹配的采样率
    language_code="zh-TW"
)

# 使用异步识别以处理较大的音频文件
operation = client.long_running_recognize(config=config, audio=audio)

print("Waiting for operation to complete...")
response = operation.result(timeout=10000)  # 设置适当的超时时间

# 转换为文本
text = " ".join([result.alternatives[0].transcript for result in response.results])
print(text)

In [ ]:
# 将文本内容保存到文件中
with open("transcription.txt", "w") as file:
    file.write(text)

print("Transcription saved in 'transcription.txt'.")

In [9]:
import spacy
from spacy.lang.zh import Chinese
from spacy.lang.zh.stop_words import STOP_WORDS
from string import punctuation

In [10]:
# 加载中文模型
nlp = spacy.load("zh_core_web_sm")

# 读取文本文件
with open("transcription.txt", "r") as file:
    text = file.read()

# 使用spaCy处理文本
doc = nlp(text)

# 获取停用词列表
stopwords = list(STOP_WORDS)
punctuations = punctuation

# 文本分句和预处理
sentences = [sentence.text.strip() for sentence in doc.sents]
sentence_tokens = [sentence for sentence in doc.sents if sentence.text.strip() not in stopwords and sentence.text.strip() not in punctuations]

# 计算句子的重要性
sentence_scores = {}
for sent in sentence_tokens:
    for word in sent:
        if word.text.lower() in stopwords or word.text.lower() in punctuations:
            continue
        if sent not in sentence_scores.keys():
            sentence_scores[sent] = 1
        else:
            sentence_scores[sent] += 1

# 基于句子数量计算摘要长度，但至少要有1个句子
summary_length = max(1, int(len(sentence_tokens) * 0.3))

# 选取分数最高的N个句子作为摘要
summary = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:summary_length]

# 显示摘要
summary_text = ' '.join([str(sentence) for sentence in summary])
print(summary_text)

裡面佛教六腳通通找到而且他的螺絲起子支援電動手動兩什麼是配偶參道便是讓你能夠根據需求自由切換什麼時候眼鏡手機組桌椅櫃子都能搞定在上則則預購2024旗艦組合還有優惠的千元枕上書家的手環 俄羅斯想見不用擔心弄到一定程度如果你還沒有看過什麼電影至少能在這個章節可以先斟酌觀賞他的那個新增提出處除三害這個電影的是香港導演黃金鼠在台灣的第一部作品比如何世說新語當中週處除三害的典故主要的在敘述由阮經天詩演的黑幫殺手陳桂林戶是否為自己罹患的末期肺癌決定有的生命的最後階段流明後是產出比自己排名高的兩名通緝犯因此講他的一連串的大屠殺隊到中最經典場景的就是陳桂林學習新興嶺社的螢幕他為了尋找排行第一的通緝犯了一度被靈少女感化差點受放下成見愁恨石化他發現教主題是一個騙錢站著柱子上的他槍殺了教主打歌 正常的血型和還誕生了許多黏土跟二次創作影片的光通講的我們就是那個感覺到這部電影充滿的是也是應該還有暴力與野獸的我們印像中午中國的定義審查很排斥著東西啊他到底是怎麼樣設定上面的那個進入中國的市場可不是一件容易的事如果不是兩個可怕的電影通常有兩種方式可以出口到中國燈照片跟麥片有分長片有這台灣的這邊放沒有完全的賣出版權何時跟中國的代理商合作讓他們可以發相應改變方的可以打到總票房收入的參數和解不過中國有限制有這種進口的分帳變得每年總共只能有34部分因為呢他噴的額度都沒好萊塢的大便想說台灣電影就比較能爭取到這種名的真的都選擇用買斷電的方式進入中國市場買斷的差別是台灣製片方會把飯吃完你出去最後 預防攝護怎樣他們都無法成本推週處除三害人走的就是買斷片方式給他去年就已經在台灣跟香港上映過也上網的袋子然後來這間公司有曾經他們其實呢是根中國代理商保理分帳指具體的數字信解綁辮頭痛的話就很好的是透過哪一種方式所有出口到中國的電影同面對就是嚴格的審查制度任何小在中國上映的電影都必須要通過中國廣播電視總局的審查審查了下雪電影審查委會負責這個我也會很神秘有沒有公佈據點名單只知道總共有36個人來自各行各業很會審查所有要上映的電影如果裡面的有他們認為的煽動抗爭危害國家安全的內容不管你是中國本土啊不是好萊塢大片對被要求刪除或修改另外中國也沒有分立制度所有定都是何 達觀賞最絕對不會出現暴力色情會宣揚邪教的內容過去能夠順利過世的台灣的通常呢只想想見你電影版這些年我們一起追的女孩子類的青春愛情片要任意外的同步的審查這部電影上面有許多血腥暴

In [11]:
print("句子数量：", len(sentences))
print("过滤后的句子数量：", len(sentence_tokens))
print("摘要长度：", summary_length)
print("文本内容：", text[:500])  # 打印部分文本内容检查
print("句子数量：", len(sentences))
print("过滤后的句子数量：", len(sentence_tokens))
print("摘要长度：", summary_length)
print("摘要内容：", summary_text)

句子数量： 3
过滤后的句子数量： 3
摘要长度： 1
文本内容： 你知道有不同的你在台灣上映的時候開放普通單半年後在中國上映票房紀錄報復嗎今天就是我們一起來了就處除三害吧後他好事執行處除三害這麼定了嗎記得今晚想吃不犯罪電影入圍的七個提名屯10月正式上映的時候在台灣的分的票房卻只有4710度報處有抄襲正一輩一部分的觀眾體質不過今年三月他的中國上有休連續使用間拿下來票房冠軍政治壓過了同期上映的好萊塢大作戰鬥門的肌肉為呢他的總票房已經突破了5.31人民幣多台灣訂第二高的路徑是1比悲傷更悲傷的故事而是中國最大的電影評分網站 日本上面的就處除三害還得老爸點1的高分子有人說跟著不定你想要撥春節檔的國產電影像是小學生寫的作業為什麼週處除三害這種口味有點重的飯的電影可以突破重重審查制度設定在中國上映和解還讓中國人艾成這個樣子呢在這之前項技能工商服務時間的時候找到對的東西嗎那就別全制霸電動螺絲起子裡面佛教六腳通通找到而且他的螺絲起子支援電動手動兩什麼是配偶參道便是讓你能夠根據需求自由切換什麼時候眼鏡手機組桌椅櫃子都能搞定在上則則預購2024旗艦組合還有優惠的千元枕上書家的手環 俄羅斯想見不用擔心弄到一定程度如果你還沒有看過什麼電影至少能在這個章節可以先斟酌觀賞他的那個
句子数量： 3
过滤后的句子数量： 3
摘要长度： 1
摘要内容： 裡面佛教六腳通通找到而且他的螺絲起子支援電動手動兩什麼是配偶參道便是讓你能夠根據需求自由切換什麼時候眼鏡手機組桌椅櫃子都能搞定在上則則預購2024旗艦組合還有優惠的千元枕上書家的手環 俄羅斯想見不用擔心弄到一定程度如果你還沒有看過什麼電影至少能在這個章節可以先斟酌觀賞他的那個新增提出處除三害這個電影的是香港導演黃金鼠在台灣的第一部作品比如何世說新語當中週處除三害的典故主要的在敘述由阮經天詩演的黑幫殺手陳桂林戶是否為自己罹患的末期肺癌決定有的生命的最後階段流明後是產出比自己排名高的兩名通緝犯因此講他的一連串的大屠殺隊到中最經典場景的就是陳桂林學習新興嶺社的螢幕他為了尋找排行第一的通緝犯了一度被靈少女感化差點受放下成見愁恨石化他發現教主題是一個騙錢站著柱子上的他槍殺了教主打歌 正常的血型和還誕生了許多黏土跟二次創作影片的光通講的我們就是那個感覺到這部電影充滿的是也是應該還有暴力與野獸的我們印像中午中國的定義審查很排斥著東西啊他到底是怎麼樣設定上面的那個進

In [12]:
import jieba
from gensim.models import Word2Vec
from collections import Counter

# 读取文本文件
with open("transcription.txt", "r", encoding='utf-8') as file:
    text = file.read()

# 加载停用词
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = [line.strip() for line in f.readlines()]

# 向jieba字典中添加特定关键词
jieba.add_word('週處除三害')

# 使用jieba进行中文分词
words = jieba.cut(text)

# 过滤长度小于2的词和停用词
filtered_words = [word for word in words if len(word) > 1 and word not in stopwords]

# 计算词频
word_counts = Counter(filtered_words)

# 获取最常出现的10个词
top_ten = word_counts.most_common(10)

# 打印结果
print("最常出現的10個關鍵字及其出現次數：")
for word, count in top_ten:
    print(f"{word}: {count}")

# Word2Vec分析
sentences = [filtered_words]  # Word2Vec模型训练需要句子列表作为输入
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# 对每个关键词找出最相似的10个词
for word, _ in top_ten:
    print(f"\n与‘{word}’最相似的10个词及相似度：")
    similar_words = model.wv.most_similar(word, topn=10)
    for similar_word, similarity in similar_words:
        print(f"{similar_word}: {similarity}")

最常出現的10個關鍵字及其出現次數：
中國: 32
電影: 19
台灣: 9
上映: 9
審查: 8
週處除三害: 7
票房: 6
宗教: 5
觀眾: 4
制度: 4

与‘中國’最相似的10个词及相似度：
歌曲: 0.3638254702091217
如何: 0.296939879655838
額度: 0.2747850716114044
兩個: 0.271655410528183
很會: 0.25940194725990295
一種: 0.23591667413711548
看到: 0.22636424005031586
可能: 0.220107764005661
阮經天: 0.21911326050758362
36: 0.21240544319152832

与‘電影’最相似的10个词及相似度：
新增: 0.3680952191352844
作業: 0.21933704614639282
世說: 0.21906261146068573
走出: 0.217889204621315
指出: 0.21726667881011963
麥片: 0.21443912386894226
斟酌: 0.21049891412258148
三月: 0.20811666548252106
認識: 0.20420905947685242
俄羅斯想: 0.19901682436466217

与‘台灣’最相似的10个词及相似度：
一定: 0.3733067512512207
的問: 0.2774883210659027
帶貨: 0.2552635073661804
有點: 0.25457462668418884
提名: 0.24949057400226593
危害: 0.22847358882427216
國主義: 0.2209750860929489
突破: 0.21657072007656097
分享: 0.21433188021183014
主要: 0.20606854557991028

与‘上映’最相似的10个词及相似度：
那麼: 0.29570648074150085
野獸: 0.2716803252696991
螢幕: 0.2564482092857361
播放: 0.2376420944929123
這邊: 0.23532213270664215
完全: 0.2239

In [7]:
print(filtered_words)


['不同', '台灣', '上映', '開放', '普通', '半年', '中國', '上映', '票房', '紀錄', '今天', '一起', '好事', '執行', '處除', '這麼定', '記得', '今晚', '犯罪', '電影', '入圍', '七個', '提名', '10', '正式', '上映', '台灣', '票房', '只有', '4710', '報處', '抄襲', '一輩', '一部分', '觀眾', '體質', '不過', '今年', '三月', '中國', '休連續', '使用', '拿下', '票房', '政治', '壓過', '同期', '上映', '好萊塢', '大作', '戰鬥門', '肌肉', '票房', '已經', '突破', '5.31', '人民', '多台', '灣訂', '第二', '路徑', '悲傷', '悲傷', '故事', '而是', '中國', '最大', '電影', '評分', '網站', '日本', '還得', '老爸', '高分子', '有人', '不定', '想要', '撥春節', '國產電', '影像', '小學生', '作業', '週處除三害', '口味', '有點', '電影', '突破', '重重', '審查', '制度', '設定', '中國', '上映', '和解', '還讓', '中國人', '艾成', '樣子', '之前', '技能', '工商', '服務', '時間', '找到', '東西', '全制', '霸電動', '螺絲', '起子', '佛教', '六腳', '通通', '找到', '而且', '螺絲', '起子', '支援', '電動', '手動', '兩什麼', '配偶', '參道', '便是', '根據', '需求', '自由', '切換', '眼鏡手', '機組', '桌椅', '櫃子', '搞定', '上則則', '預購', '2024', '旗艦', '組合', '還有', '優惠', '千元', '枕上', '書家', '手環', '俄羅斯想', '不用', '擔心', '一定', '程度', '還沒有', '看過', '電影', '至少', '章節', '斟酌', '觀賞', '新增', '提出', '處除', '電影', '香港', '導演', '金鼠', '